In [1]:
import numpy as np
import scipy.io


In [ ]:
key_time = 50e-3
Fs = 48000

In [ ]:
keys1 = scipy.io.loadmat('key1.mat')['key1']
keys2 = scipy.io.loadmat('key2.mat')['key2']
keys3 = scipy.io.loadmat('key3.mat')['key3']
notes_list = scipy.io.loadmat('notes_eighth.mat')['notes']
freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

In [ ]:
maxi = 0

In [ ]:
print(np.shape(notes_list))
print(np.shape(freq_list))
print(np.shape(keys1))


In [ ]:
def get_intensity(cur_key,cur_freq,wave_freq):
    if cur_key == 1:
        sigma = notes_list[1]-notes_list[0]
    else:
        sigma = notes_list[cur_key-1]-notes_list[cur_key-2]
    sigma = sigma/2
    intensity = np.exp(-0.5 *((cur_freq-wave_freq)/sigma)**2)
    #print('cur', cur_freq,'wave',wave_freq)
    return intensity

In [ ]:
def picknotes(cur_key,cur_freq,init_intensity):
    j = 0
    while j < len(cur_key):
        if cur_key[j] != 0:
            
            d = {}
            d['note'] = cur_key[j]
            d['start'] = j
            while cur_key[j] != 0 and j < len(cur_key)-1:
                j += 1
            d['end'] = j - 1
            intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
            d['intensity'] = intensity * init_intensity
            notes.append(d)
        j += 1
# picknotes(keys[500])     
# print(notes)      


In [ ]:
def key2wave(key,start,end,intensity=1,maxi=0):
    wave_freq = notes_list[key-1]
    duration = (end - start + 1) * key_time
    if duration > maxi:
        maxi = duration
        print(maxi)
    t = np.arange(0,duration*Fs+1)/Fs
    wave = intensity*np.sin(2*np.pi*wave_freq*t)
    return wave,maxi


In [ ]:
notes = []
for i in range(np.shape(keys1)[0]):
    cur_freq = freq_list[i]
    cur_key1 = keys1[i]
    picknotes(cur_key1,cur_freq,1)
    cur_key2 = keys2[i]
    picknotes(cur_key2,cur_freq,1)
    cur_key3 = keys3[i]
    picknotes(cur_key3,cur_freq,1)

    


In [ ]:
subwin = np.arange(0,key_time*Fs+1)/Fs
signal = np.zeros(len(subwin)*np.shape(keys1)[1])

In [ ]:
print(len(notes))

In [ ]:
for note in notes:
    wave,maxi = key2wave(note['note'], note['start'],note['end'],note['intensity'],maxi)
    x = note['start'] * len(subwin)
    signal[x:x+len(wave)] = signal[x:x+len(wave)]+wave

In [ ]:
print(maxi)

In [ ]:
scipy.io.savemat('signal2.mat', {"data2": signal })